In [10]:
import numpy as np
import openai
import pandas as pd
import os
import sys
import time

# Add the path to the constants file to the system path
sys.path.append('../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

In [11]:
# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, 'data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'movie-ml-latest-small/merged_data.csv')
print(f'Data path: {data_path}')

# zero shot save path
ZERO_SHOT_SAVE_PATH = os.path.join(DATA_DIR, 'movie-ml-latest-small/title_large_predictions_zero_shot.csv')
print(f'Zero shot save path: {ZERO_SHOT_SAVE_PATH}')

# few shot save path
FEW_SHOT_1_OBS_SAVE_PATH = os.path.join(DATA_DIR, 'movie-ml-latest-small/title_large_1_test_predictions_few_shot.csv')
print(f'Few shot save path: {FEW_SHOT_1_OBS_SAVE_PATH}')

Rec-sys directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys
Data directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/movie-ml-latest-small/merged_data.csv
Zero shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/movie-ml-latest-small/title_large_predictions_zero_shot.csv
Few shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/movie-ml-latest-small/title_large_1_test_predictions_few_shot.csv


# Data Overview

In [12]:
# Read the data
data = pd.read_csv(data_path)

# get statistic and first few data of NUM_SAMPLES rows
data.info()
data.head(NUM_EXAMPLES)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3405 entries, 0 to 3404
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  3405 non-null   int64  
 1   imdbId   3405 non-null   int64  
 2   tmdbId   3405 non-null   float64
 3   title    3405 non-null   object 
 4   genres   3405 non-null   object 
 5   userId   3405 non-null   int64  
 6   rating   3405 non-null   float64
 7   tag      3405 non-null   object 
dtypes: float64(2), int64(3), object(3)
memory usage: 212.9+ KB


,movieId,imdbId,tmdbId,title,genres,userId,rating,tag
0,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,4.0,pixar
1,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,4.0,pixar
2,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,3.5,fun
3,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy,62,4.0,fantasy
4,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy,62,4.0,magic board game


# Zero-shot (OpenAI API)

In [8]:
%%time

predict_ratings_zero_shot_and_save(data,
                                       columns_for_prediction=['title'],
                                       user_column_name='userId',
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path=ZERO_SHOT_SAVE_PATH)

Constructed Prompt for zero-shot approach:

The prompt:
**********
How will user rate this title: Toy Story (1995)? (1 being lowest and 5 being highest) Attention! Just give me back the exact number as a result, and you don't need a lot of text.

Based on the above information, please predict user's rating for the product: (1 being lowest and 5 being highest, The output should be like: (x stars, xx%), do not explain the reason.)
**********



System Fingerprint: fp_eeff13170a

API call response: "5"
Processing item 1/3405

Details: Toy Story (1995)

Predicted Rating: 5.0 stars

------------------------------------

Constructed Prompt for zero-shot approach:

The prompt:
**********
How will user rate this title: Toy Story (1995)? (1 being lowest and 5 being highest) Attention! Just give me back the exact number as a result, and you don't need a lot of text.

Based on the above information, please predict user's rating for the product: (1 being lowest and 5 being highest, The output shou

In [4]:
evaluate_model_predictions_rmse_mae(
    data_path=ZERO_SHOT_SAVE_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='rating',
    predicted_ratings_column='zero_shot_predicted_rating'
)


TypeError: unsupported operand type(s) for -: 'float' and 'str'

In [17]:
import pandas as pd

# Read the data
data = pd.read_csv(ZERO_SHOT_SAVE_PATH)

# Display the original data types
# print("Original Data Types:")
# print(data.dtypes)
# print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
data['is_rating_float'] = pd.to_numeric(data['zero_shot_predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = data[data['is_rating_float'] == False]

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


Rows with non-float ratings:


,movieId,imdbId,tmdbId,title,genres,userId,rating,tag,zero_shot_predicted_rating,is_rating_float
2,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,3.5,fun,"(None, 'Rate limit reached for gpt-3.5-turbo-1...",False
3,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy,62,4.0,fantasy,"(None, 'Rate limit reached for gpt-3.5-turbo-1...",False
4,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy,62,4.0,magic board game,"(None, 'Rate limit reached for gpt-3.5-turbo-1...",False


In [16]:
%%time 

import pandas as pd
from constants import *
from utils import *
from CF_utils import *
from tenacity import retry, wait_random_exponential, stop_after_attempt
import openai
import time
def retry_failed_predictions(data, approach):
    column_name_map = {
        'zero-shot': 'zero_shot_predicted_rating',
        'few-shot': 'few_shot_predicted_rating',
        'CF': 'cf_predicted_rating'  # Update this based on your actual column name for CF
    }

    rating_column = column_name_map.get(approach)
    if not rating_column:
        print(f"No column name found for approach: {approach}")
        return

    data['is_rating_float'] = pd.to_numeric(data[rating_column], errors='coerce').notna()
    failed_predictions = data[data['is_rating_float'] == False]

    for idx, row in failed_predictions.iterrows():
        try:
            row_df = pd.DataFrame([row])
            if approach == 'zero-shot':
                predict_ratings_zero_shot_and_save(row_df,
                                       columns_for_prediction=['title'],
                                       user_column_name='userId',
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path="process.csv")
            elif approach == 'few-shot':
                predict_ratings_few_shot_and_save(row_df)
            elif approach == 'CF':
                predict_ratings_with_collaborative_filtering_and_save(row_df)
            else:
                print(f"Unknown approach: {approach}")
                continue

            if not row_df[rating_column].isna().all():
                data.at[idx, rating_column] = row_df.at[0, rating_column]
        except Exception as e:
            print(f"Failed to predict rating for row {idx}: {e}")

    data['is_rating_float'] = pd.to_numeric(data[rating_column], errors='coerce').notna()


retry_failed_predictions(data, 'zero-shot')


Predictions saved to process.csv
Failed to predict rating for row 2: 0
Predictions saved to process.csv
Failed to predict rating for row 3: 0
Predictions saved to process.csv
Failed to predict rating for row 4: 0
Predictions saved to process.csv
Failed to predict rating for row 5: 0
Predictions saved to process.csv
Failed to predict rating for row 6: 0
Predictions saved to process.csv
Failed to predict rating for row 7: 0
Predictions saved to process.csv
Failed to predict rating for row 8: 0
Predictions saved to process.csv
Failed to predict rating for row 9: 0
Predictions saved to process.csv
Failed to predict rating for row 15: 0
Predictions saved to process.csv
Failed to predict rating for row 16: 0
Predictions saved to process.csv
Failed to predict rating for row 17: 0
Predictions saved to process.csv
Failed to predict rating for row 18: 0
Predictions saved to process.csv
Failed to predict rating for row 19: 0
Predictions saved to process.csv
Failed to predict rating for row 25: 0


# Few-shot (OpenAI API)


+ For each user, we'll use 4 of their ratings as training data to predict ratings for the rest of their products. Finally, we'll evaluate the predictions against the actual ratings to calculate the overall RMSE and MAE.

+ The rating_history_str now includes both the title and the review text for each of the training data rows

# 1 observation per reviewer - Few-shot OpenAI

In [6]:
%%time

predict_ratings_few_shot_and_save(data,
                                      columns_for_training=['title'],
                                       columns_for_prediction=['title'],
                                       user_column_name='userId',
                                       asin_column_name='movieId', 
                                       obs_per_user=1,
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path=FEW_SHOT_1_OBS_SAVE_PATH)


Constructed Prompt for few-shot approach:

The prompt:
**********
How will user rate this title: Sin City (2005)? (1 being lowest and 5 being highest) Attention! Just give me back the exact number as a result, and you don't need a lot of text.

Here is user rating history:
* title: Corpse Bride (2005) - Rating: 2.5 stars
* title: Wedding Crashers (2005) - Rating: 4.0 stars
* title: Lord of War (2005) - Rating: 4.0 stars
* title: Cinderella Man (2005) - Rating: 4.5 stars

Based on the above information, please predict user's rating for the product: (1 being lowest and 5 being highest, The output should be like: (x stars, xx%), do not explain the reason.)
**********



System Fingerprint: fp_eeff13170a

API call response: "3.5 stars"
Processing user 1/25, item 1/9
User 336:
Rating History for Prediction:
* title: Corpse Bride (2005) - Rating: 2.5 stars
* title: Wedding Crashers (2005) - Rating: 4.0 stars
* title: Lord of War (2005) - Rating: 4.0 stars
* title: Cinderella Man (2005) - Rat

In [7]:
evaluate_model_predictions_rmse_mae(
    data_path=FEW_SHOT_1_OBS_SAVE_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='few_shot_predicted_rating'
)


RMSE: 0.9834 (95% CI: (0.7308, 1.2249)) ± 0.0025
MAE: 0.7935 (95% CI: (0.5455, 1.0455)) ± 0.0025

First few actual vs predicted ratings:
Actual: 4.0, Predicted: 3.5000
Actual: 2.0, Predicted: 2.0000
Actual: 3.5, Predicted: 4.0000
Actual: 4.0, Predicted: 3.5000
Actual: 3.5, Predicted: 3.0000


# Limitations:

The model might not fully understand the nuanced relationships between products based on titles alone. Additional context or features might be needed for more accurate predictions.
This approach might be computationally expensive and slower than traditional matrix factorization or deep learning-based recommendation models, especially for a large number of users.

# References

+ https://platform.openai.com/docs/api-reference/authentication